In [2]:
import sagemaker

from sagemaker.pytorch import PyTorch
from sagemaker.session import Session
from sagemaker.experiments.run import Run
from sagemaker.utils import unique_name_from_base

sagemaker_session = sagemaker.Session()
bucket = sagemaker_session.default_bucket()

role = sagemaker.get_execution_role()

## Train

In [ ]:
experiment_name = unique_name_from_base("captioner")
with Run(experiment_name=experiment_name, run_name="run-01", sagemaker_session=Session()) as run:
    
    estimator = PyTorch(
        role=role,
        entry_point="train.py",
        framework_version="2.0.0",
        py_version="py310",
        instance_count=1,
        instance_type="ml.g4dn.xlarge",
        keep_alive_period_in_seconds=30,
        dependencies=['requirements.txt', 'model.py', 'data.py'],
        hyperparameters={"epochs": 5, "batch-size":64, 'lr':0.0002, 'heads':8, 'nlayers':4}
    )
    
    estimator.fit({'train': f's3://{bucket}/train2017', 'test': f's3://{bucket}/val2017', 'text': f's3://{bucket}/annotations'})

## Inference